### 1. Введение

✍ Мы подошли к заключительной части раздела, посвящённого математическому анализу и оптимизации. В предыдущих модулях мы уже познакомились с функциями и их исследованием, научились находить минимальные и максимальные значения для функций, а также применять эти значения для решения реальных задач. В данном модуле мы продолжим углубляться в методы оптимизации, сравним их и обсудим, для каких случаев какие алгоритмы подходят лучше всего.

За два прошлых модуля мы узнали достаточно много важных понятий и инструментов — давайте повторим их ↓

**Задание 1.1**

Пусть прибыль в вашей компании выражается следующей функцией, которая зависит от параметра x — количества производимых товаров:

$f(x)=-x^{4}+6 x^{3}-4 x^{2}+80$

Найдите максимально возможную прибыль, которую вы можете получить, варьируя количество произведённых товаров.

In [5]:
import numpy as np
import pandas as pd
import sympy
from sympy import Symbol, S
from sympy import diff
from sympy import solveset, Eq
from sympy import *
 
x= symbols('x')
f = -x**4 + 6*x**3 - 4*x**2 + 80
fa = f.diff(x) 
sol = solve(fa, x)
sol

[0, 1/2, 4]

In [10]:
fa


-4*x**3 + 18*x**2 - 8*x

In [16]:
x = 4
eval('-x**4 + 6*x**3 - 4*x**2 + 80')

144

**Задание 1.4**

Допустим, вы хотите произвести некоторое количество товара, которое зависит от часов работы двух ключевых сотрудников следующим образом:

$f(x, y)=x^{2}+2 y^{2}$

Однако вы можете оплатить этим сотрудникам не более 20 часов работы.

Какое наибольшее количество товаров вы сможете произвести в таком случае?

Помните, что количество часов работы должно быть целым, поэтому, прежде чем вычислять итоговый результат, округлите часы работы до целого.

In [20]:
from sympy import *
x,y,w = symbols(' x y w' )
g = x**2 +  2*y**2
print('Целевая функция для аргументов a и b :\n f = ', gamma)
q = x + y - 20
print('Функция ограничений: ', q,'= 0')
f = x**2 +  2*y**2 + w*(x + y - 20)
print('Функция Лагранжа :\n ',f)
fx = f.diff(x)
print('df/dx =',fx,'= 0')
fy = f.diff(y)
print('df/dy =',fy,'= 0')
fw = f.diff(w)
print('df/dw =',fw,'= 0')
sols = solve([fx,fy,fw],x,y,w)
print('Стационарная точка M(x,y):\n',float(sols[x]),',',float(sols[y]))

x = round(float(sols[x]), 0)
y = round(float(sols[y]), 0)
eval('x**2 +  2*y**2')

Целевая функция для аргументов a и b :
 f =  gamma
Функция ограничений:  x + y - 20 = 0
Функция Лагранжа :
  w*(x + y - 20) + x**2 + 2*y**2
df/dx = w + 2*x = 0
df/dy = w + 4*y = 0
df/dw = x + y - 20 = 0
Стационарная точка M(x,y):
 13.333333333333334 , 6.666666666666667


267.0

### ЭТОТ МОДУЛЬ МЫ ПОСВЯТИМ ИЗУЧЕНИЮ НОВЫХ МЕТОДОВ ОПТИМИЗАЦИИ:

- рассмотрим, какие вариации существуют у уже известного вам алгоритма градиентного спуска, и узнаем, в чём суть **обратного распространения ошибки**;
- познакомимся с **методом Ньютона** и **квазиньютоновскими методами BFGS и L-BFGS**;
- разберём область применения задач линейного программирования и попрактикуемся в их решении;
- узнаем, что такое **метод отжига** и **метод координатного спуска**.

Из предыдущего модуля вы помните, что в каких-то методах оптимизации мы использовали лишь значение функции, где-то — считали градиент, а где-то — находили матрицы производных. Эти особенности дают возможность поделить все алгоритмы на три группы:

- методы нулевого порядка (их работа основана на оценке значений самой целевой функции в разных точках);
- методы первого порядка (при работе они используют первые производные в дополнение к информации о значении функции);
- методы второго порядка (для них необходимо оценивать и значение функции, и значение градиента, и гессиан (матрицу Гессе).

**Примечание**. Иногда в литературе можно встретить термины «оракул первого порядка» или «оракул нулевого порядка». Так обозначают компоненты алгоритма, которые находят информацию на каждом шаге для метода соответствующего порядка.

В данном модуле мы будем рассматривать методы разных порядков и практически в каждом юните будем практиковаться в их применении. В конце мы обобщим полученные знания и сравним, какие методы и в каких случаях показывают наилучшие результаты.

→ Важно отметить, что некоторые алгоритмы оптимизации мы пока не сможем использовать по прямому назначению: мы опробуем их на известных вам моделях линейной или логистической регрессии или на функциях одной или нескольких переменных, но основная сфера их применения — это искусственные нейронные сети. Тем не менее, понимание всего пула алгоритмов поможет вам в будущем без проблем подбирать необходимый метод вне зависимости от того, задачу какой сложности вы решаете: минимизируете простейшую функцию или обучаете нейронную сеть со сложной архитектурой.

___________________________

### 2. Градиентный спуск: применение и модификации